<a href="https://colab.research.google.com/github/ilkzeyden/text-to-flowchart/blob/main/T5_JSONDAN_MODEL_E%C4%9E%C4%B0T%C4%B0M%C4%B0_(17_05_25).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install transformers datasets

In [ ]:
import os
import pandas as pd
import json
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict

In [ ]:
import os
import pandas as pd
import json
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict

#############################################
# 1. Excel'den JSON'a Dönüştürme Fonksiyonu
#############################################
import pandas as pd
import json

# 📂 Excel dosyasını oku (Dosya adını değiştir)
file_path = r'/content/drive/My Drive/akış-diyagramı-örnekleri-verileri.xlsx'  # Dosyanın adını güncelle

df = pd.read_excel(file_path)

# 📌 Gerekli sütunları seçelim
columns_needed = ["Akış ID", "Adım No", "Adım Türü", "Adım Açıklaması", "Evet Durumu (Sonraki Adım)", "Hayır Durumu (Sonraki Adım)"]
df = df[columns_needed]

# 📌 Akış ID'ye göre verileri gruplandıralım
data_by_flow = {}

for _, row in df.iterrows():
    flow_id = row["Akış ID"]
    step_number = row["Adım No"]

    # Çıktıyı oluştur
    output = {
        "Adım No": step_number,
        "Açıklama": row['Adım Açıklaması'],
        "Adım Türü": row["Adım Türü"],
        "Sonraki Adım (Evet)": row["Evet Durumu (Sonraki Adım)"] if pd.notna(row["Evet Durumu (Sonraki Adım)"]) else None,
        "Sonraki Adım (Hayır)": row["Hayır Durumu (Sonraki Adım)"] if pd.notna(row["Hayır Durumu (Sonraki Adım)"]) else None
    }

    # Girdi-Çıktı çiftini oluştur
    data_point = {
        "input": f"Adım No: {step_number}, Açıklama: {row['Adım Açıklaması']}, Adım Türü: {row['Adım Türü']}",
        "output": output
    }

    # Akış ID'sine göre veriyi grupla
    if flow_id not in data_by_flow:
        data_by_flow[flow_id] = []
    data_by_flow[flow_id].append(data_point)

# 📂 JSON dosyası olarak kaydet
output_file = "hazirlanmis_veri.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data_by_flow, f, ensure_ascii=False, indent=4)

print(f"✅ JSON formatında veri hazır! Kaydedilen dosya: {output_file}")

#############################################
# 3. Model ve Tokenizer Yükleme
#############################################
custom_cache_dir = "./huggingface_cache"
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name, cache_dir=custom_cache_dir)
model = T5ForConditionalGeneration.from_pretrained(model_name, cache_dir=custom_cache_dir)

#############################################
# 4. Tokenizasyon & Preprocessing
#############################################
def preprocess_function(examples):
    inputs = examples["input"]
    targets = examples["output"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

#############################################
# 5. Eğitim Ayarları
#############################################
training_args = TrainingArguments(
    output_dir="./t5_flowchart_output",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    num_train_epochs=7,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=3,
    fp16=torch.cuda.is_available()
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return {"dummy_metric": 0.0}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

#############################################
# 6. Eğitimi Başlatma ve Test Örneği Üzerinde Çıktı Alma
#############################################
if __name__ == "__main__":
    trainer.train()

    test_example = dataset["test"][0]["input"]
    input_ids = tokenizer.encode(test_example, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(input_ids, max_length=512)
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("📝 Girdi:", test_example)
    print("🎯 Çıktı:")
    print(predicted_text)

#  wandb şifresi = abda9f461371669c2516207660e00058a83e1e09

✅ JSON formatında veri hazır! Kaydedilen dosya: hazirlanmis_veri.json


Map:   0%|          | 0/535 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/75 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-08494b5a591e>:104: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss


📝 Girdi: 1: Öğrenci, staj başvuru formunu sistem üzerinden doldurur ve gerekli belgeleri yükler.
2: Öğrenci başvuru formu ve belgeleri veri tabanına kaydedilir.
3: Belgelerde eksiklik veya hata var mı?
4: Eksik veya hatalı belgelerin düzeltilmesi için öğrenciye otomatik e-posta gönderilir.
5: Öğrenci, eksik belgeleri tamamlar ve yeniden yükler.
6: Eksiksiz başvuru formları değerlendirilmek üzere staj komisyonuna iletilir.
7: Staj komisyonu başvuruyu onayladı mı? Onaylandıysa öğrenciye onay maili atılır, onaylanmadıysa reddedilme maili atılır
8: Onaylanan başvuru için öğrenciye staj kabul mektubu e-posta ile gönderilir.
9: Başvurusu reddedilen öğrenciye, gerekçeler ile birlikte e-posta gönderilir.
10: Staj başvurusu kabul edilen öğrencilerin bilgileri ve belgeleri veri tabanında güncellenir.
11: Staj başlangıç tarihi ve süreç takibi için sistemden öğrenciye bilgilendirme yapılır.
12: Süreç sona erer.
🎯 Çıktı:
1: renci, staj başvuru formu sistem üzerind doldurur ve gerekli belgeleri yükl